# Navigating data ontologies

```{tip}
Open this tutorial in Google Colab for interactive viewing.
```

In [1]:
# @title Install AlphaGenome

# @markdown Run this cell to install AlphaGenome.
from IPython.display import clear_output
! pip install alphagenome
clear_output()

## Setup and imports

In [3]:
from alphagenome import colab_utils
from alphagenome.models import dna_client
import pandas as pd

## Interactively view output metadata

First, we load the model.

In [5]:
%pip install dotenv


  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]
Note: you may need to restart the kernel to use updated packages.


In [7]:
from dotenv import load_dotenv
import os
load_dotenv()

ALPHA_GENOME_API_KEY = os.getenv("ALPHA_GENOME_API_KEY")

if not ALPHA_GENOME_API_KEY:
    raise ValueError("ALPHA_GENOME_API_KEY not found in .env file")

In [8]:
dna_model = dna_client.create(ALPHA_GENOME_API_KEY)

output_metadata = dna_model.output_metadata(
    dna_client.Organism.HOMO_SAPIENS
).concatenate()

Click `Filter` on the upper right hand side of the interactive dataframe and type a cell or tissue name like "brain" into the `Search by all fields box` to find the `ontology_curie` term corresponding to a tissue and output type of interest:

In [9]:
output_metadata

,name,strand,Assay title,ontology_curie,biosample_name,biosample_type,biosample_life_stage,data_source,endedness,genetically_modified,nonzero_mean,output_type,gtex_tissue,histone_mark,transcription_factor
0,CL:0000084 ATAC-seq,.,ATAC-seq,CL:0000084,T-cell,primary_cell,adult,encode,paired,False,0.739741,OutputType.ATAC,NaN,NaN,NaN
1,CL:0000100 ATAC-seq,.,ATAC-seq,CL:0000100,motor neuron,in_vitro_differentiated_cells,adult,encode,paired,False,0.273136,OutputType.ATAC,NaN,NaN,NaN
2,CL:0000236 ATAC-seq,.,ATAC-seq,CL:0000236,B cell,primary_cell,adult,encode,paired,False,4.700081,OutputType.ATAC,NaN,NaN,NaN
3,CL:0000623 ATAC-seq,.,ATAC-seq,CL:0000623,natural killer cell,primary_cell,adult,encode,paired,False,0.938715,OutputType.ATAC,NaN,NaN,NaN
4,CL:0000624 ATAC-seq,.,ATAC-seq,CL:0000624,"CD4-positive, alpha-beta T cell",primary_cell,adult,encode,paired,False,4.365206,OutputType.ATAC,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,ENCSR182QNJ,-,PRO-cap,EFO:0001099,Caco-2,cell_line,NaN,encode,NaN,False,14.002803,OutputType.PROCAP,NaN,NaN,NaN
8,ENCSR740IPL,-,PRO-cap,EFO:0002067,K562,cell_line,NaN,encode,NaN,False,15.765458,OutputType.PROCAP,NaN,NaN,NaN
9,ENCSR797DEF,-,PRO-cap,EFO:0002819,Calu3,cell_line,NaN,encode,NaN,False,12.281321,OutputType.PROCAP,NaN,NaN,NaN
10,ENCSR801ECP,-,PRO-cap,CL:0002618,endothelial cell of umbilical vein,primary_cell,NaN,encode,NaN,False,13.973692,OutputType.PROCAP,NaN,NaN,NaN


In [ ]:
output_metadata.to_csv('output_metadata.csv')

### How many tracks are there per output type?

In [11]:
# Count human tracks
human_tracks = (
    dna_model.output_metadata(dna_client.Organism.HOMO_SAPIENS)
    .concatenate()
    .groupby('output_type')
    .size()
    .rename('# Human tracks')
)

# Count mouse tracks
mouse_tracks = (
    dna_model.output_metadata(dna_client.Organism.MUS_MUSCULUS)
    .concatenate()
    .groupby('output_type')
    .size()
    .rename('# Mouse tracks')
)

pd.concat([human_tracks, mouse_tracks], axis=1).astype(pd.Int64Dtype())

,# Human tracks,# Mouse tracks
output_type,,
OutputType.ATAC,167,18
OutputType.CAGE,546,188
OutputType.DNASE,305,67
OutputType.RNA_SEQ,667,173
OutputType.CHIP_HISTONE,1116,183
OutputType.CHIP_TF,1617,127
OutputType.SPLICE_SITES,4,4
OutputType.SPLICE_SITE_USAGE,734,180
OutputType.SPLICE_JUNCTIONS,367,90


Note that `PROCAP` outputs are not available for mouse.